In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from google.colab import files
import numpy as np
import shutil
import math
import cv2
import os

In [2]:
def Timestamp_To_Seconds(timestamp):
    # Split the timestamp into hours, minutes, and seconds
    parts = timestamp.split(':')
    hours = int(parts[0])
    minutes = int(parts[1])
    seconds = int(parts[2])

    # Convert timestamp to seconds
    total_seconds = hours * 3600 + minutes * 60 + seconds

    return total_seconds

In [3]:
def Extract_Frame(video_path, output_path, timestamp):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Could not open the video.")
        return None

    # Convert timestamp to seconds
    timestamp_sec = Timestamp_To_Seconds(timestamp)

    # Set the video's position to the desired timestamp
    cap.set(cv2.CAP_PROP_POS_MSEC, timestamp_sec * 1000)

    # Read the frame at the desired timestamp
    ret, frame = cap.read()

    if not ret:
        print("Error: Could not read frame.")
        return None

    cv2.imwrite(output_path, frame, [cv2.IMWRITE_PNG_COMPRESSION, 0])

    # Release the video capture object
    cap.release()

In [4]:
def Get_Lane_Area(image, vertices):
  mask = np.zeros_like(image)

  if len(image.shape) > 2:
        channel_count = image.shape[2]
        ignore_mask_color = (255,) * channel_count
  else:
        ignore_mask_color = 255

  cv2.fillPoly(mask, vertices, ignore_mask_color)
  masked_image = cv2.bitwise_and(image, mask)

  return masked_image, mask

In [5]:
def Draw_Lines(image, lines, color=[255, 0, 0], thickness=12):
  for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(image, (x1, y1), (x2, y2), color, thickness)

In [6]:
def Hough_Transformation(image, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(image, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    Draw_Lines(line_image, lines, thickness=2)

    return line_image, lines

In [7]:
def Merge_Image(line_image, initial_image):
  # Check if images are None
  if line_image is None or initial_image is None:
      raise ValueError("One or both input images are None.")

  # Check if images have the same dimensions
  if line_image.shape != initial_image.shape:
      print(line_image.shape)
      print(initial_image.shape)

  # Check if images have the same data type
  if line_image.dtype != initial_image.dtype:
      raise ValueError("Input images must have the same data type.")
  return cv2.addWeighted(initial_image, 1.0, line_image, 1.0, 0.0)

In [8]:
def Plot_Image(image, title):
  plt.title(title)
  if len(image.shape) == 3:
      plt.imshow(image)
  else:
      plt.imshow(image, cmap='gray')

In [9]:
def Detect_Lane(image, plot_image=False,
                kernel_size=5, canny_low_threshold=50, canny_high_threshold=150,
                hough_rho=1, hough_theta=np.pi/180, hough_threshold=15,
                hough_min_line_len=20, hough_max_line_gap=500):

    # Convert to gray scale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    gray_blur = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)

    # Canny Edge Detection
    edges = cv2.Canny(gray_blur, canny_low_threshold, canny_high_threshold)

    # Concentrate the location of edge detection
    image_shape = image.shape
    vertices = np.array([[(100,image_shape[0]),
                          (image_shape[1]/2-30, image_shape[0]/2+50),
                          (image_shape[1]/2+30, image_shape[0]/2+50),
                          (image_shape[1]-20, image_shape[0])]], dtype=np.int32)

    masked_edges, mask = Get_Lane_Area(edges, vertices=vertices)

    # Detect lines using Hough transform on an edge detected image
    lines_image, lines = Hough_Transformation(masked_edges,
                                  rho=hough_rho, theta=hough_theta, threshold=hough_threshold,
                                  min_line_len=hough_min_line_len, max_line_gap=hough_max_line_gap)

    # Merge 'original' image with 'lines' image
    result = Merge_Image(lines_image, image)

    # Plot the images
    if plot_image:
        plt.figure(figsize=[16, 9])
        for i, img in enumerate(['gray', 'gray_blur', 'edges', 'mask', 'masked_edges', 'lines_image', 'result']):
            Plot_Image(eval(img), img,)
            plt.show()
    return result, lines


In [10]:
def Connect_Lane_line(lines, imshape):
  try:
      # Vectorise the lines representation
      lines = np.squeeze(lines)

      # Compute the slope and length of line segments
      m = (lines[:,3] - lines[:,1]) / (lines[:,2] - lines[:,0])
      lengths = np.sqrt((lines[:,3] - lines[:,1])**2 + (lines[:,2] - lines[:,0])**2)

      # Get rid of outliers
      mask = np.abs(m) > 0.5
      lines, m, lengths = lines[mask], m[mask], lengths[mask]

      # Perform classification
      l_lines, r_lines = lines[m < 0, :], lines[m > 0, :]
      l_m, r_m = m[m < 0], m[m > 0]
      l_lengths, r_lengths = lengths[m < 0], lengths[m > 0]

      # Get sorted args for the classified line segments based on their lengths
      # so we can determine the best candidates for lane detection.
      l_sorted_args, r_sorted_args = np.argsort(l_lengths), np.argsort(r_lengths)

      # Compute the slope and intercepts from the best candidates
      n = -5
      l_avgm, r_avgm = l_m[l_sorted_args][n::].mean(), r_m[r_sorted_args][n:].mean()

      l_x, l_y = np.concatenate([l_lines[:,0],l_lines[:,2]]), np.concatenate([l_lines[:,1],l_lines[:,3]])
      r_x, r_y = np.concatenate([r_lines[:,0],r_lines[:,2]]), np.concatenate([r_lines[:,1],r_lines[:,3]])

      l_b = l_y - (l_avgm * l_x)
      r_b = r_y - (r_avgm * r_x)
      l_avgb, r_avgb = l_b[l_sorted_args][n:].mean(), r_b[r_sorted_args][n:].mean()

      # Determine the L&R lane endpoints
      y1 = imshape[0]/1.5
      l_x1 = (y1 - l_avgb) / l_avgm
      y2 = imshape[0]
      l_x2 = (y2 - l_avgb) / l_avgm

      r_x1 = (y1 - r_avgb) / r_avgm
      r_x2 = (y2 - r_avgb) / r_avgm

      connected_lines = np.int32([[[l_x1,y1,l_x2,y2], [r_x1,y1,r_x2,y2]]])
      return connected_lines
  except Exception as e:
      print(e)

In [ ]:
def Process_Video(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)

    if not cap.isOpened():
        print("Error: Không thể mở video.")
        return

    # Get video information
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Create an output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    # Process each frame
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Draw lane lines on a frame
        result, lines = Detect_Lane(frame)

        # Write processed frame into the ouput video
        out.write(result)

    # Free resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [ ]:
lane_image_path = 'lane_img/'
result_image_path = 'img_result/'
test_video_path = 'test_video/'
result_video_path ='video_result/'

if not os.path.exists(lane_image_path):
    os.mkdir(lane_image_path)

if not os.path.exists(result_image_path):
    os.mkdir(result_image_path)

if not os.path.exists(test_video_path):
    os.mkdir(test_video_path)

if not os.path.exists(result_video_path):
    os.mkdir(result_video_path)

In [ ]:
uploaded = files.upload()

for filename in uploaded.keys():
  shutil.move(filename, os.path.join(test_video_path, filename))

Saving test_lane_video2.mp4 to test_lane_video2.mp4


In [ ]:
input_video_path = os.path.join(test_video_path, input('Enter test video name: '))
output_video_path = os.path.join(result_video_path, input('Enter result video name: '))

# Xử lý video và tạo video kết quả
Process_Video(input_video_path, output_video_path)

In [ ]:
# Extract a frame from the video
'''
video_name = 'test_lane_video2.mp4'
image_name = 'lane_image1.jpg'
video_path = os.path.join(test_video_path, video_name)
output_path = os.path.join(lane_image_path, image_name)

timestamp = '0:0:8'
Extract_Frame(video_path, output_path, timestamp)
'''

In [ ]:
# read all image from 'lane_img/' and append them to a list
'''
image_names = os.listdir(lane_image_path)

image_list = []
for image_name in image_names:
  image_path = os.path.join(lane_image_path, image_name)
  if os.path.isfile(image_path):
      image = mpimg.imread(image_path)
      image_list.append(image)
for image in image_list:
  print(image.shape)
  plt.imshow(image)
  plt.show()
'''

In [ ]:
# Process all image from a list and save result into 'img_result'
'''
for _, imgname in enumerate(image_names):
    img_path = os.path.join(lane_image_path, imgname)
    if os.path.isfile(img_path):
        img = mpimg.imread(img_path)
        result,_ = Detect_Lane(img, plot_image=True)
        mpimg.imsave(os.path.join(result_image_path, imgname), result)
    else:
        print(f"Skipping {img_path} as it's not a file.")
'''